In [1]:
from thermo import *
from fluids import *
from thermo.chemical import Chemical

In [2]:
import math
from fluids.units import *

Given:  Air at 65 psig and 110 degF is flowing through 75' of 1" Schedule 40 pipe at a rate of 100 standard cubic feet per minute (scfm)

Find:  The pressure drop in pounds per square inch and the velocity in feet per minute at both up stream and downstream gauges.

# The solution from Crane 410

Refer to table on Crane 410 B12, look up 100 scfm and across to 1" pipe -> pressure drop of 2.21 psi.
This 2.21 psi is in reference to "Pressure drop per 100 feet of schedule 40 pipe at 100 psig and 60 degF".

The pressure drop need to correct for pressure, length, and temperature.

L/100 for length correction
(100 + 14.7)/(P + 14.7)*(460 + t)/520 where P is the inlet or average gauge pressure (psi), and t (degF)

Length, Pressure and Temperature correction for pressure drop

In [3]:
L = 75*u.feet
P = 65*u.psi
t = 110*u.degF
deltaP = 2.21*u.psi*L.magnitude/100*(100 + 14.7)/(P.magnitude + 14.7)*(460 + t.magnitude)/520

'{:.2f}'.format(deltaP)

'2.61 pound_force_per_square_inch'

To find the velocity, the rate of flow in cubic feet per minute at flowing conditions must be determined.

The correction from Crane 410, B-12 is:
14.7/(14.7+P))*(460 + t)/520, where P is inlet or average gauge pressure (psi) and t is temperature (degF)

In [4]:
#At upstream gauge
qm_given = 100*u.feet**3/u.min
qm_up = qm_given*14.7/(14.7+P.magnitude)*(460 + t.magnitude)/520

#At downstream gauge
qm_down = qm_given*14.7/(14.7+(P-deltaP).magnitude)*(460 + t.magnitude)/520

qm_up, qm_down

(<Quantity(20.217643084644337, 'foot ** 3 / minute')>,
 <Quantity(20.90343030044916, 'foot ** 3 / minute')>)

In [5]:
#Area
d = nearest_pipe(NPS=1*u.inch, schedule='40')[1].to(u.inch) #inner pipe dimension
A = (math.pi*(d**2)/4).to(u.feet**2)
'{:.3f}'.format(A)

'0.006 foot ** 2'

In [6]:
#At upstream gauge
V_up = qm_up/A

#At downstream gauge
V_down = qm_down/A

'velocity upstream is {:.2f}, velocity downstream is {:.2f}'.format(V_up, V_down)


'velocity upstream is 3369.78 foot / minute, velocity downstream is 3484.09 foot / minute'

# Using the fluids module

In [7]:
L = (75*u.feet).to(u.meter)
t = (110*u.degF).to(u.degK)
D = d.to(u.meter)
q = qm_given.to(u.meter**3/u.sec)
A = A.to(u.meter**2)
P1 = (65*u.psi+14.7*u.psi).to(u.Pa)

S1 = Stream('air', T=t.magnitude, P=P1.magnitude, Q = q)
rho = S1.rho*u.kg/u.meter**3
m = rho*q
V = q/A

e = roughness_Farshad('Carbon steel, bare', D=D)
Re = S1.Reynolds(D=D, V=V)

fd = friction_factor(Re=Re.magnitude, eD=e.magnitude/D.magnitude)


P2 = isothermal_gas(rho=rho, fd=fd, P1=P1.magnitude, D=D.magnitude, L=L.magnitude, m=m.magnitude)

((P1.magnitude-P2)*u.Pa).to(u.psi)
rho

TypeError: 549512.1562655185 has no quantity

Can't get the fluids module and the thermo module to output the correct answer.